In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import bloomfilter
import block
import ecc
import helper
import merkleblock
import network
import script
import tx

In [ ]:
from helper import hash256
bit_field_size = 10  # <1>
bit_field = [0] * bit_field_size
h = hash256(b'hello world')  # <2>
bit = int.from_bytes(h, 'big') % bit_field_size  # <3>
bit_field[bit] = 1  # <4>
print(bit_field)

In [ ]:
from helper import hash256
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):  # <1>
    h = hash256(item)
    bit = int.from_bytes(h, 'big') % bit_field_size
    bit_field[bit] = 1
print(bit_field)

### Exercise 1

Calculate the Bloom Filter for 'hello world' and 'goodbye' using the `hash160` hash function over a bit field of 10.

In [ ]:
# Exercise 1

from helper import hash160

bit_field_size = 10
bit_field = [0] * bit_field_size
items = (b'hello world', b'goodbye')
# loop through each item
    # hash160 the item
    # interpret hash as a Big-Endian integer and mod by bit_field_size
    # set that bit in bit_field to 1
# print the bit_field

In [ ]:
from helper import hash256, hash160
bit_field_size = 10
bit_field = [0] * bit_field_size
for item in (b'hello world', b'goodbye'):
    for hash_function in (hash256, hash160):  # <1>
        h = hash_function(item)
        bit = int.from_bytes(h, 'big') % bit_field_size
        bit_field[bit] = 1
print(bit_field)

In [ ]:
from helper import murmur3  # <1>
from bloomfilter import BIP37_CONSTANT  # <2>
field_size = 2
num_functions = 2
tweak = 42
bit_field_size = field_size * 8
bit_field = [0] * bit_field_size
for phrase in (b'hello world', b'goodbye'):  # <3>
    for i in range(num_functions):  # <4>
        seed = i * BIP37_CONSTANT + tweak  # <5>
        h = murmur3(phrase, seed=seed)  # <6>
        bit = h % bit_field_size
        bit_field[bit] = 1
print(bit_field)

### Exercise 2

Given a Bloom Filter with size=10, function count=5, tweak=99, what are the bytes that are set after adding these items? (use `bit_field_to_bytes` to convert to bytes)

* `b'Hello World'`
* `b'Goodbye!'`

In [ ]:
# Exercise 2

from bloomfilter import BloomFilter, BIP37_CONSTANT
from helper import bit_field_to_bytes, murmur3

field_size = 10
function_count = 5
tweak = 99
items = (b'Hello World',  b'Goodbye!')
# calculate the bitfield size
# create an empty bit field
# loop through items
    # loop through function count
        # calculate the seed
        # get the murmur3 hash of the item using the seed
        # mod by the bitfield size
        # set the bit
# convert the bit field to bytes
# print the bytes in hex

### Exercise 3

Write the `add` method for `BloomFilter`

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_add`

In [ ]:
# Exercise 3

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_add"))

### Exercise 4

Write the  `filterload` payload from the `BloomFilter` class.

#### Make [this test](/edit/code-ch12/bloomfilter.py) pass: `bloomfilter.py:BloomFilterTest:test_filterload`

In [ ]:
# Exercise 4

reload(bloomfilter)
run(bloomfilter.BloomFilterTest("test_filterload"))

### Exercise 5

Write the `serialize` method for the `GetDataMessage` class.

#### Make [this test](/edit/code-ch12/network.py) pass: `network.py:GetDataMessageTest:test_serialize`

In [ ]:
# Exercise 5

reload(network)
run(network.GetDataMessageTest("test_serialize"))

In [ ]:
from bloomfilter import BloomFilter
from helper import decode_base58
from merkleblock import MerkleBlock
from network import FILTERED_BLOCK_DATA_TYPE, GetHeadersMessage, GetDataMessage, HeadersMessage, SimpleNode
from tx import Tx
last_block_hex = '00000000000538d5c2246336644f9a4956551afb44ba47278759ec55ea912e19'
address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
h160 = decode_base58(address)
node = SimpleNode('tbtc.programmingblockchain.com', testnet=True, logging=False)
bf = BloomFilter(size=30, function_count=5, tweak=90210)  # <1>
bf.add(h160)  # <2>
node.handshake()
node.send(bf.filterload())  # <3>
start_block = bytes.fromhex(last_block_hex)
getheaders = GetHeadersMessage(start_block=start_block)
node.send(getheaders)  # <4>
headers = node.wait_for(HeadersMessage)
getdata = GetDataMessage()  # <5>
for b in headers.blocks:
    if not b.check_pow():
        raise RuntimeError('proof of work is invalid')
    getdata.add_data(FILTERED_BLOCK_DATA_TYPE, b.hash())  # <6>
node.send(getdata)  # <7>
found = False
while not found:
    message = node.wait_for(MerkleBlock, Tx)  # <8>
    if message.command == b'merkleblock':
        if not message.is_valid():  # <9>
            raise RuntimeError('invalid merkle proof')
    else:
        for i, tx_out in enumerate(message.tx_outs):
            if tx_out.script_pubkey.address(testnet=True) == address:  # <10>
                print('found: {}:{}'.format(message.id(), i))
                found = True
                break

### Exercise 6

Get the current testnet block ID, send yourself some testnet coins, find the UTXO corresponding to the testnet coin _without using a block explorer_, create a transaction using that UTXO as an input and broadcast the `tx` message on the  testnet network.

In [ ]:
# Exercise 6

import time

from block import Block
from bloomfilter import BloomFilter
from ecc import PrivateKey
from helper import hash256, little_endian_to_int, encode_varint, read_varint, decode_base58, SIGHASH_ALL
from merkleblock import MerkleBlock
from network import (
    GetDataMessage,
    GetHeadersMessage,
    HeadersMessage,
    NetworkEnvelope,
    SimpleNode,
    TX_DATA_TYPE,
    FILTERED_BLOCK_DATA_TYPE,
)
from script import p2pkh_script, Script
from tx import Tx, TxIn, TxOut

last_block_hex = ''  # FILL THIS IN

secret = little_endian_to_int(hash256(b''))  # FILL THIS IN
private_key = PrivateKey(secret=secret)
addr = private_key.point.address(testnet=True)
h160 = decode_base58(addr)

target_address = 'mwJn1YPMq7y5F8J3LkC5Hxg9PHyZ5K4cFv'
target_h160 = decode_base58(target_address)
target_script = p2pkh_script(target_h160)
fee = 5000  # fee in satoshis


# connect to tbtc.programmingblockchain.com in testnet mode
# create a bloom filter of size 30 and 5 functions. Add a tweak that you like
# add the h160 to the bloom filter
# complete the handshake
# load the bloom filter with the filterload command

# set start block to last_block from above
# send a getheaders message with the starting block

# wait for the headers message
# store the last block as None
# initialize the GetDataMessage
# loop through the blocks in the headers
    # check that the proof of work on the block is valid
    # check that this block's prev_block is the last block
    # add a new item to the get_data_message
    # should be FILTERED_BLOCK_DATA_TYPE and block hash
    # set the last block to the current hash
# send the getdata message

# initialize prev_tx and prev_index to None
# loop while prev_tx is None 
    # wait for the merkleblock or tx commands
    # if we have the merkleblock command
        # check that the MerkleBlock is valid
    # else we have the tx command
        # set the tx's testnet to be True
        # loop through the tx outs
            # if our output has the same address as our address we found it
                # we found our utxo. set prev_tx, prev_index, and transaction
# create the TxIn
# calculate the output amount (previous amount minus the fee)
# create a new TxOut to the target script with the output amount
# create a new transaction with the one input and one output
# sign the only input of the transaction
# serialize and hex to see what it looks like
# send this signed transaction on the network
# wait a sec so this message goes through with time.sleep(1) 
# now ask for this transaction from the other node
# create a GetDataMessage
# ask for our transaction by adding it to the message
# send the message
# now wait for a Tx response
# if the received tx has the same id as our tx, we are done!